In [8]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from langgraph.checkpoint.memory import InMemorySaver

class ChatState(TypedDict):
    user_input: str
    intent: Literal["greeting", "question", "complaint", "other"]
    response: str

def identify_intent(state: ChatState):
    text = state["user_input"].lower()
    if "hello" in text or "hi" in text:
        return {"intent": "greeting"}
    elif "?" in text:
        return {"intent": "question"}
    elif "problem" in text or "not working" in text:
        return {"intent": "complaint"}
    else:
        return {"intent": "other"}

def handle_greeting(state: ChatState):
    return {"response": "Hello! How can I help you today?"}

def handle_question(state: ChatState):
    return {"response": "That's a great question! Let me find the answer."}

def handle_complaint(state: ChatState):
    return {"response": "I'm sorry to hear that. Can you provide more details?"}

def handle_other(state: ChatState):
    return {"response": "Can you please clarify your request?"}

def route_intent(state: ChatState):
    return state["intent"]

graph = StateGraph(ChatState)
graph.add_node("identify_intent", identify_intent)
graph.add_node("handle_greeting", handle_greeting)
graph.add_node("handle_question", handle_question)
graph.add_node("handle_complaint", handle_complaint)
graph.add_node("handle_other", handle_other)

graph.add_edge(START, "identify_intent")
graph.add_conditional_edges("identify_intent", route_intent, {
    "greeting": "handle_greeting",
    "question": "handle_question",
    "complaint": "handle_complaint",
    "other": "handle_other"
})
graph.add_edge("handle_greeting", END)
graph.add_edge("handle_question", END)
graph.add_edge("handle_complaint", END)
graph.add_edge("handle_other", END)
checkpoint_saver = InMemorySaver()
workflow = graph.compile(checkpointer=checkpoint_saver)

In [10]:
config1 = {"configurable":{"thread_id":"1"}}
workflow.invoke({
    "user_input": "what is python?"
},config=config1)

{'user_input': 'what is python?',
 'intent': 'question',
 'response': "That's a great question! Let me find the answer."}

In [ ]:
workflow.get_state(config1) # This will return final state of the workflow

StateSnapshot(values={'user_input': 'what is python?', 'intent': 'question', 'response': "That's a great question! Let me find the answer."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074953-82f9-620b-8002-4dbb5582f1c3'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-08T20:21:09.007617+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074953-82f9-620a-8001-7b1e552f75e9'}}, tasks=(), interrupts=())

In [13]:
list(workflow.get_state_history(config1))  # This will return all keys in the state

[StateSnapshot(values={'user_input': 'what is python?', 'intent': 'question', 'response': "That's a great question! Let me find the answer."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074953-82f9-620b-8002-4dbb5582f1c3'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-08T20:21:09.007617+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074953-82f9-620a-8001-7b1e552f75e9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'user_input': 'what is python?', 'intent': 'question'}, next=('handle_question',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074953-82f9-620a-8001-7b1e552f75e9'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-08T20:21:09.007617+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f074953-82